In [1]:
import pandas as pd

from hardware.opentrons.opentrons_api import OpentronsAPI

from hardware.cameras import PendantDropCamera

from hardware.sensor.sensor_api import SensorAPI
from hardware.opentrons.formulater import Formulater

from hardware.opentrons.configuration import Configuration
from analysis.plots import Plotter

from utils.logger import Logger
from utils.load_save_functions import load_settings
from utils.search_containers import get_well_id_concentration, get_well_id_solution
from analysis.active_learning import ActiveLearner
from analysis.models import szyszkowski_model

from hardware.opentrons.droplet_manager import DropletManager


opentrons_api = OpentronsAPI()

opentrons_api.initialise()

config = Configuration(opentrons_api=opentrons_api)

labware = config.load_labware()

containers = config.load_containers()

pipettes = config.load_pipettes()

left_pipette = pipettes["left"]

right_pipette = pipettes["right"]

pendant_drop_camera = PendantDropCamera()
sensor_api = SensorAPI()
plotter = Plotter()

formulater = Formulater(
    left_pipette=left_pipette,
    right_pipette=right_pipette,
    containers=containers,
)

droplet_manager = DropletManager(
    left_pipette=left_pipette,
    containers=containers,
    pendant_drop_camera=pendant_drop_camera,
    opentrons_api=opentrons_api,
    plotter=plotter,
)

c:\Users\pimda\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-24 17:54:27,963 - INFO - Protocol already uploaded, using existing protocol.
2025-03-24 17:54:35,243 - INFO - Run created succesfully (ID: 62104968-5061-4fdc-a0ff-55725a250da8).
2025-03-24 17:54:37,898 - INFO - All custom labware definitions added.
2025-03-24 17:54:39,022 - INFO - Labware loaded successfully.
2025-03-24 17:54:39,056 - INFO - Containers loaded successfully.
2025-03-24 17:54:43,178 - INFO - Pipettes loaded successfully.


In [2]:
opentrons_api.home()

2025-03-24 17:41:34,260 - INFO - Robot homed.


In [4]:
left_pipette.drop_tip()

2025-03-24 17:42:39,215 - INFO - Left pipette dropped tip into trash.


In [ ]:
drop_parameters = drop_parameters = {
    "drop_volume": 11,
    "max_measure_time": 30,
    "flow_rate": 1,
}
st_t, img_ana = droplet_manager.measure_pendant_drop(source=containers['7A1'], drop_parameters=drop_parameters)
# droplet_manager._make_pendant_drop(source=containers['7A1'], drop_volume=11, flow_rate=1, drop_count=1)

In [ ]:
droplet_manager._return_pendant_drop(source=containers['7A1'], drop_volume=11)

In [ ]:
import numpy as np
np.mean([item[1] for item in st_t])
# left_pipette.return_needle()

In [ ]:
droplet_manager._make_pendant_drop(
    source=containers["7A1"],
    drop_volume=11,
    flow_rate=1,
    drop_count=1
)

In [ ]:
droplet_manager._return_pendant_drop(
    source=containers['7A1'],
    drop_volume=11
)

In [ ]:
pendant_drop_camera.stop_capture()
pendant_drop_camera.stop_stream()

In [ ]:
drop_parameters = drop_parameters = {
    "drop_volume": 11,
    "max_measure_time": 30,
    "flow_rate": 1,
}
st_t, img_ana = droplet_manager.measure_pendant_drop(
    source=containers["7A1"], drop_parameters=drop_parameters
)

In [ ]:
formulater.wash(repeat=3)

In [ ]:
left_pipette.return_needle()

In [ ]:
left_pipette.clean_on_sponge()

In [ ]:
offset = {
    'x': 0,
    'y': 0,
    'z': -23.4
}
left_pipette.move_to_well(container=containers['drop_stage'], offset=offset)

In [ ]:
left_pipette.pick_up_tip()

In [ ]:
opentrons_api.home()

In [ ]:
left_pipette.aspirate(
    volume=10,
    source = containers["3A1"],
    flow_rate=1
)

In [ ]:
left_pipette.dispense(volume=10, destination=containers["3A1"], flow_rate=1)

In [ ]:
well_id_water = get_well_id_solution(containers=containers, solution_name="water")
well_id_trash = get_well_id_solution(containers=containers, solution_name="trash")
if not right_pipette.has_tip:
    right_pipette.pick_up_tip()

right_pipette.aspirate(
    volume=300,
    source=containers[well_id_water],
    touch_tip=True
)
right_pipette.dispense(
    volume=300,
    destination=containers["3A1"],
    touch_tip=True
)

In [ ]:
left_pipette.aspirate(
    volume=20,
    source=containers["7A1"],
    flow_rate=1
)

In [ ]:
left_pipette.mixing(
    container=containers["3A1"],
    mix = ("after", 20, 5)
)

In [ ]:
left_pipette.volume = 0

In [ ]:
right_pipette.aspirate(
    volume=300,
    source=containers["3A1"],
    touch_tip=True,
)
right_pipette.dispense(
    volume=300,
    destination=containers[well_id_trash],
    update_info=False
)

In [ ]:
right_pipette.drop_tip()

In [ ]:
left_pipette.clean_on_sponge()

In [ ]:
left_pipette.return_needle()

In [ ]:
droplet_manager.measure_pendant_drop(
    source=containers["7A5"],
    drop_parameters={"drop_volume": 6, "max_measure_time": 60, "flow_rate": 1}
)